In [20]:
import numpy as np
import pandas as pd
import matplotlib as plt

## Are the cases for Texas increasing decreasing or stable for the last week of 2020 data?

In [21]:
# First I need to import the super_covid_dataset so I can manipulate it.
super_covid = pd.read_csv('./data/super_covid_data.csv')

# Once I've done that, create a dataframe that only contains the data for Texas.
texas_data = super_covid[super_covid['State'] == 'TX']

# Total Population being at the end is annoying so I'm moving it to be beside State FIPS
populations = texas_data.pop('population')
texas_data.insert(4, populations.name, populations)

# I don't need to work with all of the texas data so I'm going to cull most of it
data_2020 = texas_data.filter(like='2020')
cases_2020 = data_2020.filter(like='cases')
week_data = cases_2020.iloc[:,-7:]
texas_data = texas_data.iloc[:,0:5]

# Now that I've culled the data I'm going to put it back together just in case I need it later.
texas_data = pd.concat([texas_data, week_data], axis=1)

# We're going to sum the weekly data to make my life easier.
week_data.loc['totals'] = week_data.sum()
week_data

C:\Users\rulef\AppData\Local\Temp\ipykernel_18708\3714780109.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_data.loc['totals'] = week_data.sum()


,2020-12-25_cases,2020-12-26_cases,2020-12-27_cases,2020-12-28_cases,2020-12-29_cases,2020-12-30_cases,2020-12-31_cases
2547,0,0,0,0,0,0,0
2548,4270,4270,4270,4270,4505,4510,4538
2549,1361,1361,1361,1361,1361,1405,1416
2550,5186,5186,5186,5186,5186,5370,5489
2551,691,691,700,700,709,722,730
...,...,...,...,...,...,...,...
2798,759,759,759,759,759,775,779
2799,1372,1372,1379,1379,1394,1404,1419
2800,904,904,912,912,929,958,958
2801,858,858,858,858,861,871,876


### The totals row increases significantly every day of the week that we interrogated.  So cases in Texas were increasing in the last week of 2020.

# Implement the merge for your enrichment data.

In [22]:
bls_data = pd.read_excel('./data/allhlcn20.xlsx')
bls_data

,Area\nCode,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,US000,US,0.0,0,10,2020,A,Nation,NaN,U.S. TOTAL,Total Covered,"10 Total, all industries",NaN,10487687,139103773,8905606771748,1231,64021,1.00,1.00
1,US000,US,0.0,1,10,2020,A,Nation,NaN,U.S. TOTAL,Federal Government,"10 Total, all industries",NaN,61310,2928036,250251253418,1644,85467,1.00,1.00
2,US000,US,0.0,2,10,2020,A,Nation,NaN,U.S. TOTAL,State Government,"10 Total, all industries",NaN,70059,4592925,302037717914,1265,65762,1.00,1.00
3,US000,US,0.0,3,10,2020,A,Nation,NaN,U.S. TOTAL,Local Government,"10 Total, all industries",NaN,171044,13642800,776054840247,1094,56884,1.00,1.00
4,US000,US,0.0,5,10,2020,A,Nation,NaN,U.S. TOTAL,Private,"10 Total, all industries",NaN,10185275,117940012,7577262960169,1236,64247,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62887,C4974,NaN,NaN,5,1024,2020,A,MSA,NaN,"Yuma, AZ MSA",Private,1024 Professional and business services,NaN,453,6682,317633827,914,47535,0.69,0.56
62888,C4974,NaN,NaN,5,1025,2020,A,MSA,NaN,"Yuma, AZ MSA",Private,1025 Education and health services,NaN,434,7720,434779038,1083,56320,0.73,1.06
62889,C4974,NaN,NaN,5,1026,2020,A,MSA,NaN,"Yuma, AZ MSA",Private,1026 Leisure and hospitality,NaN,325,5656,127248900,433,22496,0.93,1.16
62890,C4974,NaN,NaN,5,1027,2020,A,MSA,NaN,"Yuma, AZ MSA",Private,1027 Other services,NaN,214,1202,43609156,698,36275,0.64,0.76


### Now that I've imported the information I need to edit 'Area\nCode' until it maps well to super_covid.  Once it does that I can change the column name and merge on that column

In [23]:
bls_cull = bls_data[bls_data['Area Type'] == 'County']

# Change the Super Covid and BLS datasets to use integers for FIPS for easy handling
bls_cull['Area\nCode'] = bls_cull['Area\nCode'].astype('int64')

# Rename the column so that I can merge
bls_cull = bls_cull.rename(columns={'Area\nCode':'countyFIPS'})

# Merge The Dataframes
covid_employment = pd.merge(super_covid, bls_cull, on=['countyFIPS'], how='left')

covid_employment

C:\Users\rulef\AppData\Local\Temp\ipykernel_18708\2012344957.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bls_cull['Area\nCode'] = bls_cull['Area\nCode'].astype('int64')


,countyFIPS,County Name,State,StateFIPS,2020-01-22_cases,2020-01-23_cases,2020-01-24_cases,2020-01-25_cases,2020-01-26_cases,2020-01-27_cases,...,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,Total Covered,"10 Total, all industries",NaN,910.0,10794.0,465745049.0,830.0,43147.0,1.00,1.00
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,Federal Government,"10 Total, all industries",NaN,11.0,90.0,5105909.0,1088.0,56575.0,0.40,0.39
3,1001,Autauga County,AL,1,0,0,0,0,0,0,...,State Government,"10 Total, all industries",NaN,19.0,563.0,28429201.0,972.0,50518.0,1.58,1.80
4,1001,Autauga County,AL,1,0,0,0,0,0,0,...,Local Government,"10 Total, all industries",NaN,32.0,1859.0,77184261.0,799.0,41529.0,1.76,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53403,56045,Weston County,WY,56,0,0,0,0,0,0,...,Private,1024 Professional and business services,NaN,24.0,53.0,3418357.0,1242.0,64599.0,0.16,0.17
53404,56045,Weston County,WY,56,0,0,0,0,0,0,...,Private,1025 Education and health services,NaN,34.0,183.0,6181801.0,649.0,33734.0,0.50,0.43
53405,56045,Weston County,WY,56,0,0,0,0,0,0,...,Private,1026 Leisure and hospitality,NaN,22.0,180.0,2276050.0,244.0,12680.0,0.86,0.59
53406,56045,Weston County,WY,56,0,0,0,0,0,0,...,Private,1027 Other services,NaN,13.0,29.0,1287178.0,859.0,44642.0,0.45,0.64
